# Probabilistic Matrix Factorization 
## Introduction
In this notebook, we implement probabilistic matrix factorization (PMF) on movie rating dataset.

Matrix factorization is a method to decompose a matrix into multiplication of matrices (usually two matrices). A matrix can represent a dyadic data. Matrix factorization is a way to better understand the components (dyads), and use them in further tasks. The goal is to find those unknown matrices.

Probabilistic matrix factorization (PMF) is a probabilistic extension of matrix factorization where unknown parameters are described with their probability distributions. Similar to matrix factorization, the goal of PMF is to learn embeddings from dyadic/relational data (each matrix entry is a dyad, e.g., user-item rating, document-word count, user-user link). If the matrix has missing values, by learning decomposition based on observed entries, missing values can be pre dicted. This technique is useful in variety of applications such as recommender systems/collaborative filtering, link prediction in social network, and so on. In this project, are going to use PMF in collaborating filtering. The goal of collab orative filtering is to infer user preferences for items given a large but incomplete collection of preferences for many users. If we let each row of the original matrix represent a user, and each column represent a product, we can interpret the entries of the matrix to be the rating given to a product by a user.

By learning latent features for users and movies using the seen ratings, we are able to predict the ratings for the unseen entries. 

### Model 
For more clarification, we show the model in the context of movie ratings. Let $R$ be the matrix of ratings which is partially (and sparsely) filled by users' ratings for movies. Let us assume that there are $N$ users and $M$ movies in total. The goal is to find a decomposition for $R$ such that:
$$R \approx U^T V$$
where $U$ is a $d\times N$ matrix describing users and $V$ is a $d\times M$ matrix describing movies. 

Figure below shows an illustration of this model where each column of $U$ and $V$ learn a representation for the users and movies respectively.

<img src="matrix_fact.png",style="max-width:100%; width: 50%">


Given feature vectors for $U_i$ user and $V_j$, the distribution of the corresponding rating is:

$$R(i,j) \sim N(U_i^TV_j , \sigma^2)$$
$$U_i \sim N(0, \sigma_u^2 I) ~~~~ V_j \sim N(0, \sigma_v^2 I)$$

The graphical model of it is:

<img src="graph-model.png",height: 20>

Our objective is to find $U$ and $V$ such that they have the maximum probabillity to generate seen ratings $R$. This method is called Maximum a Posteriori (MAP) estimation. Note that $\lambda$ is a tuning parameter resulting in more stability. Let $D$ represent set of all observed data such that: $D~: ~\{(i,j) ~| ~R(i,j)~is ~given\}$. Then the objective is:

$$\underset{U,V}{\max} ~ \log p(U,V|R) $$
$$\underset{U,V}{\max} ~ L(U,V)=-\frac{1}{2\sigma^2} \sum_{(i,j)\in D} {(R_{ij} - U_i^{\intercal} V_j)}^2 - \frac{\lambda}{2} \sum_{i=1}^{M} {||U_i||}^2 - \frac{\lambda}{2} \sum_{j=1}^{N} {||V_j||}^2$$

The optimization algorithm is as follows:

- Initialize $U^{(0)}$ and $V^{(0)}$
- For $t = 1:T$
    - $U^{(t)}_i = {\left[ \lambda I + \sum_{j:(i,j)\in D} \frac{1}{\sigma^2} {V^{(t-1)}_j}^{\intercal}V^{(t-1)}_j  \right]}^{-1} \left(\sum_{j:(i,j)\in D} \frac{1}{\sigma^2}R_{ij} ~ V^{(t-1)}_j \right)$
    - $V^{(t)}_j = {\left[ \lambda I + \sum_{i:(i,j)\in D} \frac{1}{\sigma^2} {U^{(t)}_i}^{\intercal} U^{(t)}_i  \right]}^{-1} \left(\sum_{i:(i,j)\in D} \frac{1}{\sigma^2}R_{ij} ~ U^{(t)}_i \right)	
$

    - End for.
    
    
Now, the first step is to import the data.    


You need to load your data, and put it in a matrix called $R$. Make sure to convert the ratings into floats - not categorical variables. The goal for you is to complete the matrix and find the missing entries. To make calculations easier, create a new matrix with 3 columns. Put the row indices (user ids) in the first row, and column indices (movie ids) in second column, and let the third column show the ratings.
Here is an implementation of it for MovieLens dataset.

In [21]:
## Load your data here...
# example with MovieLens data:
from numpy import *
import random
import pandas as pd
prefer = []
for line in open('data/u.data.txt', 'r'):  
    (userid, movieid, rating, ts) = line.split('\t')  
    uid = int(userid)
    mid = int(movieid)
    rat = float(rating)
    prefer.append([uid, mid, rat])
    #data = array(prefer)

In [22]:
data = array(prefer)
N = len(np.unique(data[:,0])) # number of users
M = len(np.unique(data[:,1])) # number of movies
print("In this dataset, there are {} users and {} movies\objects.".format(N,M))

In this dataset, there are 943 users and 1682 movies\objects.


In [23]:
print("there are {} given ratings total.".format(len(data)))
      
print("Here is the first 10 rows in my processesed data")
data[:10]

there are 100000 given ratings total.
Here is the first 10 rows in my processesed data


array([[ 196.,  242.,    3.],
       [ 186.,  302.,    3.],
       [  22.,  377.,    1.],
       [ 244.,   51.,    2.],
       [ 166.,  346.,    1.],
       [ 298.,  474.,    4.],
       [ 115.,  265.,    2.],
       [ 253.,  465.,    5.],
       [ 305.,  451.,    3.],
       [   6.,   86.,    3.]])

Now you need to run your algorithm. Note that at each sep, you need to update all of the movies' features as well as all of the users' features. You can set lambda = 0.1, and variances to be 1. Note that these parameter choices are up to you, and feel free to tune them the way you want. You can set the number of iterations to be 100.

In [24]:
## Initialize U and V randomly. Note that U should be d by N and V should be d by M. 

To make the calculations easier, you should keep track of some values at each step. Note that to update vector for a user, you need to have access to all the movies rated by that user (the same for the movies; to update vector for a movie, you need the id of all of the users that have rated that movie.)
Since these don't change, we suggest you to construct a dictionary for each user as key, and id of movies rated by that user to be the values (as a list). You should do the same for movies.
These are only some suggestions, feel free to use any method that you like. :)

In [36]:
## you can construct two dictionaries one for the user and one for the movies here.
## hint: you can use defaultdict

# from collections import defaultdict
#
# u_list = defaultdict(list)
# for i, j in zip(data[:,0],data[:,0]):
#    u_list[i].append(j)
#    
# m_list = defaultdict(list)
# for i, j in zip(data[:,0],data[:,1]):
#    m_list[i].append(j)


Again, to update feature vector of a user, you need to compute ${V^{(t-1)}_j}^{\intercal}V^{(t-1)}_j$. We suggest you to ceate an array for all of the movies, and update these values at the end of each iteration. That way, you just need to look up the dictionary for the movie ids that a user has rated, and use those indices of your constructed vector.
You need to do the same for the movies.

In [26]:
## Constructing arrays that will make your life easier in the implementation of the algorithm ....

In [27]:
## Implementation of the algorithm ...

Now that you found the optimal U and V matrices, you can predict for the missing ratings for users. For example, show 10 movies with the highest predicted ratings for 5 of the users of your choice.


In [28]:
## e.g.: If you were a movie recommending system, what would you recommend to user number 19 to watch that hasn't watched before?

### Reference:
1- Mnih, A., & Salakhutdinov, R. (2007). Probabilistic matrix factorization. In Advances in neural information processing systems (pp. 1257-1264). https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf
